In [1]:
from urllib import parse

import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel("data/FN_Training_Set.xlsx")

In [6]:
df.Content = df.Content.astype(str)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()


print("Extracting tf-idf features for NMF...")
tfidf = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=20000)

x_tfidf = tfidf.fit_transform(df.Content)

print("Extracting tf features for LDA...")
tf = CountVectorizer(max_df=0.95, min_df=2, max_features=20000)
x_tf = tf.fit_transform(df.Content)

n_topics = 30
n_top_words = 20
nmf = NMF(n_components=n_topics, random_state=1,alpha=.1, l1_ratio=.5)
nmf.fit(x_tfidf)

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(x_tf)

print("\nTopics in LDA model:")
tf_feature_names = tf.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Extracting tf-idf features for NMF...
Extracting tf features for LDA...

Topics in NMF model:
Topic #0:
да се от за не че по са си това ще като но които най много може има той до
Topic #1:
робева нешка гимнастика художествена художествената събрание че леля каза треньорката писано тя това заслужават ще отбор аз ванга 25 камък
Topic #2:
анонимните корея война за групата предстоящата заедно филипините подготвят южна сащ корейския опустошителна брутална налице признаци екологично региона жестока бърза
Topic #3:
да подарък се ви такъв портмоне подарява подаръци кърпички празно неприятности вещ отрицателна предмет предмети сълзи ръкавици око трябва добре
Topic #4:
благовещение благовец риба се блага задължително благовест празникът празнува правите ден трапезата яде имате необходимо през пари този благовеста благой
Topic #5:
ще море черно потоп предрича от се порои балканският потънат родопите земята нов потъне загинат много свещена ванга полуостров планина
Topic #6:
ще караджов има сериозн

In [16]:
x = lda.transform(x_tf)

In [17]:
x.shape

(2815, 30)

In [37]:
df_train, df_test = train_test_split(df, test_size=0.1)

In [32]:
clf = LinearSVC()
clf.fit(x[:2000], df.click_bait_score[:2000])
clf.score(x[2000:], df.click_bait_score[2000:])

0.57177914110429451

In [33]:
clf.coef_

array([[ 0.18889406,  0.48177274,  0.05314431,  0.29585665, -0.00581791,
         0.51432634,  0.06751052,  0.05314431,  0.28088832,  0.05314431,
         0.41771862,  0.05314431, -0.01603148,  0.13484227,  0.05314431,
        -0.0974764 ,  0.34861122,  0.41776952, -0.43845479, -0.27263414,
         0.05314431, -0.26184092, -0.56506981, -0.60194653, -0.07238538,
        -1.10869766, -0.32740197,  0.03097911,  1.17993345, -0.14773544]])

In [31]:
x_random = np.random.randn(*x.shape)
clf = LinearSVC()
clf.fit(x_random[:2000], df.click_bait_score[:2000])
clf.score(x_random[2000:], df.click_bait_score[2000:])

0.57177914110429451